# ReAct implementation using guidance

# Wikipedia tool

In [14]:
import wikipediaapi
import wikipedia

wiki = wikipediaapi.Wikipedia('WikiReactGuidance (selfint@gmail.com)', 'en')

In [15]:
from typing import Callable
from transformers import PreTrainedTokenizerFast
import collections
from functools import partial

def Tree():
    return collections.defaultdict(Tree)

def build_tree(sections, tree = None) -> Tree:
    if tree is None:
        tree = Tree()

    for s in sections:
        tree[s.title] = (s.text, build_tree(s.sections))

    return dict(tree)


def chunk_tokens(
    tokens: list[str],
    chunk_size: int,
    chunk_overlap: int,
    is_subword: Callable[[str], bool],
) -> list[list[str]]:
    chunks = []

    while len(tokens) > 0:
        # reduce chunk size to not end with subword
        chunk_len = chunk_size
        while (chunk_len + 1 < len(tokens)) and is_subword(tokens[chunk_len + 1]):
            chunk_len -= 1
        assert chunk_len > 0, "got empty chunk"

        chunk_tokens = tokens[:chunk_len]
        chunks.append(chunk_tokens)

        new_start = chunk_len
        if new_start > len(tokens):
            break

        # try to reduce overlap to not cut subwords
        overlap = chunk_overlap if chunk_overlap < chunk_len else 0
        while is_subword(tokens[new_start - overlap]):
            overlap -= 1
            if overlap == 0:
                print("failed to prevent subword cut:", tokens[new_start - 1], tokens[new_start])
                break

        tokens = tokens[new_start - overlap:]

    return chunks


def build_text_chunks(
    text: str,
    title: str,
    title_path: list[str],
    tokenizer: PreTrainedTokenizerFast,
    chunk_size: int,
    chunk_overlap: int = 0,
) -> list[dict]:
    # build header
    header = "\n".join(
        f"{'#' * (i + 1)} {t}" for i, t in enumerate(title_path + [title])
    ) + "\n"

    # split tokens into chunk sized chunks
    text_chunk_size = chunk_size - len(tokenizer.tokenize(header))
    text_chunks_tokens = chunk_tokens(
        tokenizer.tokenize(text),
        text_chunk_size,
        chunk_overlap,
        is_subword = lambda t: not t.startswith("▁")
    )

    # build chunks
    chunks = [
        {
            "text": header + tokenizer.decoder.decode(text_chunk),
            "meta": {
                "title": title,
                "title_path": title_path
            }
        }
        for text_chunk in text_chunks_tokens
    ]

    for c in chunks:
        if 'is documented as' in c["text"]:
            print(c["text"])

    return chunks

def build_tree_chunks(
    tree: Tree,
    title_path: list[str],
    tokenizer: PreTrainedTokenizerFast,
    chunk_size: int,
    chunk_overlap: int = 0,
) -> list[dict]:
    assert chunk_size > chunk_overlap * 2, f"overlap must be < {chunk_size // 2=}"
    _build_text_chunks = partial(
        build_text_chunks,
        tokenizer=tokenizer,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    chunks = []
    title_path = title_path or []

    for k, (text, subtrees) in tree.items():
        chunks += _build_text_chunks(text, k, title_path)

        # build subtree chunks
        for subtitle, (subtext, subtree) in subtrees.items():
            chunks += _build_text_chunks(subtext, subtitle, title_path + [k])
            chunks += build_tree_chunks(
                subtree, title_path, tokenizer, chunk_size, chunk_overlap,
            )

    return chunks

In [16]:
from FlagEmbedding import FlagReranker
import torch

torch.set_default_device("mps")
reranker = FlagReranker('./models/BAAI/bge-reranker-large')

In [68]:
from tqdm.auto import tqdm

def search_wikipedia(query, top_pages, top_chunks, chunk_size, chunk_overlap, thought = None):
    thought = thought or query
    results = wikipedia.search(query, top_pages, False)
    pages = [wiki.page(r) for r in results]
    chunks = []

    for page in tqdm(pages, desc="Chunking page contents"):
        tree = build_tree(page.sections)
        page_chunks = build_tree_chunks(
            tree, [page.title], reranker.tokenizer, chunk_size, chunk_overlap
        )
        for c in page_chunks:
            c["meta"]["page"] = page
        chunks += page_chunks

    scores = reranker.compute_score(
        [[query, c["text"]] for c in chunks]
    )

    best = [a[0] for a in sorted(zip(chunks, scores), key=lambda a: a[1], reverse=True)]
    return best[:top_chunks]

In [18]:
from textwrap import wrap

def print_chunks(chunks):
    print(
        "\n--\n".join(
            "\n".join(
                wrap(
                    c["text"],
                    width=100,
                    break_long_words=False,
                    replace_whitespace=False,
                )
            ) for c in chunks
        )
    )

In [19]:
print_chunks(search_wikipedia(
    "How much wood could a wood chuck chuck if a wood chuck could chuck wood?",
    1,
    3,
    256,
    64
))

Chunking page contents:   0%|          | 0/1 [00:00<?, ?it/s]

# How much wood would a woodchuck chuck
## Origin
The origin of the phrase is from a 1902 song "The Woodchuck Song", written by Robert Hobart Davis for Fay Templeton in the musical The Runaways. The lyrics became better known in a 1904 version of the song written by Theodore Morse, with a chorus of "How much wood would a woodchuck chuck if a woodchuck could chuck wood?", which was recorded by Ragtime Roberts, in 1904.The tongue-twister is documented as "folklore" in 1972 at Farmington, Michigan. It is used in the title of Werner Herzog's 1976 film How Much Wood Would a Woodchuck Chuck, a documentation of the World Livestock Auctioneer Championship in New Holland, Pennsylvania.
# How much wood would a woodchuck chuck
## Answers
A traditional, if nonsensical, "response" to the
question is: "A woodchuck would chuck as much wood as a woodchuck could chuck if a woodchuck could
chuck wood". Other—similarly unhelpful—responses include "So much wood would a woodchuck chuck as a
woodchuck would

# Load Model

In [8]:
from transformers import AutoModelForCausalLM

model_path = "./models/ehartford/dolphin-2.1-mistral-7b"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Guidance

In [10]:
from guidance.llms import Transformers

class ChatMlModel(Transformers):
    llm_name = "chatml-model"

    @staticmethod
    def role_start(role, *args, **kwargs):
       return  f"<|im_start|>{role}\n"

    @staticmethod
    def role_end(role, *args, **kwargs):
        return "<|im_end|>"

start to install package: redis



[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


successfully installed package: redis
start to install package: redis-om



[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


successfully installed package: redis-om


In [11]:
llm = ChatMlModel(model=model, tokenizer=tokenizer)

In [12]:
import guidance
guidance.llm = llm

In [117]:
def search(query: str, thought: str) -> str:
    results = search_wikipedia(
        query=query, thought=thought, top_pages=3, top_chunks=1, chunk_size=256, chunk_overlap=64
    )

    return "\n---\n".join(c["text"] for c in results)

react = guidance(
    search=search,
    template='''\
{{#system~}}
You are WikiBot, an assistant that can answer questions about the world \
by using information available in Wikipedia, and by explaining your thoughts \
and actions to the user. Given a question, here are the components of your response:
1. Think - explain to the user your thought process behind how you are trying to \
figure out the answer.
2. Act - here you either Search for information, if more information is needed to \
answer the question. Or you Finish and provide the final answer, if you are confident \
you know figure it out. You always start this step with "Search" or "Finish", \
followed by square brackets containing the search string (Search[search string]) \
or the final answer (Finish[final answer]).
3. Observe - only occurs if you Searched for information in step 2. Here you show \
the user the relevant information you found in Wikipedia to answer your search from \
step 2.

These components repeat until you choose the Finish action and provide the final \
answer.
{{~/system}}
{{#user~}}
Before we start, let's have a practice round.
Seven Brief Lessons on Physics was written by an Italian physicist that has worked \
in France since what year?
{{~/user}}
{{#assistant~}}
# Thought
I need to search Seven Brief Lessons on  Physics, find its author, then find \
when the author has worked in France since.

# Act
Search[Seven Brief Lessons on Physics]

# Observation
Seven Brief Lessons on Physics (Italian: Sette brevi lezioni di fisica) \
is a short book by the Italian physicist Carlo Rovelli. Originally published in \
Italian in...

# Thought
The author of Seven Brief Lessons on Physics is Carlo Rovelli. I need to search \
Carlo Rovelli next and find when he has word in France since.

# Act
Search[Carlo Rovelli]

# Observation
Carlo Rovelli is an Italian theoretical physicist and writer who has worked in \
Italy, the United States and, since 2000, in France.[1] He is also currently a \
Distinguished Visiting Research Chair at the Perimeter Institute...

# Thought
Carlo Rovelli has worked in France since 2000. So the answer is 2000.

# Act
Finish[2000]
{{~/assistant}}
{{#user~}}
Great job! Now for the real question:
{{question}}
{{~/user}}
{{#assistant~}}
{{#geneach 'chain' stop=False~}}
# Thought
{{gen 'this.thought' max_tokens=512 stop="\n# Act"}}
# Act
{{select 'this.act' options=['Search', 'Finish']}}[{{gen 'this.act_content' max_tokens=512 stop="]"}}]
{{#if this.act == 'Search'}}
# Observation
{{#block hidden=True~}}
{{set 'this.observation' (search query=this.act_content thought=this.thought)}}
{{/block~}}
{{this.observation}}
{{else}}
{{break}}
{{/if}}
{{/geneach}}
{{~/assistant}}\
''')

In [121]:
result = react(
    question="How to calculate the limit of a fraction, when both the numerator and the denominator approach 0?",
    # question="Where was the final battle between Caesar and Pompey?",
)

Stop program system You are WikiBot, an assistant that can answer questions about the world by using information available in Wikipedia, and by explaining your thoughts and actions to the user. Given a question, here are the components of your response:
1. Think - explain to the user your thought process behind how you are trying to figure out the answer.
2. Act - here you either Search for information, if more information is needed to answer the question. Or you Finish and provide the final answer, if you are confident you know figure it out. You always start this step with "Search" or "Finish", followed by square brackets containing the search string (Search[search string]) or the final answer (Finish[final answer]).
3. Observe - only occurs if you Searched for information in step 2. Here you show the user the relevant information you found in Wikipedia to answer your search from step 2.

These components repeat until you choose the Finish action and provide the final answer. user Before we start, let's have a practice round.
Seven Brief Lessons on Physics was written by an Italian physicist that has worked in France since what year? assistant # Thought
I need to search Seven Brief Lessons on Physics, find its author, then find when the author has worked in France since.

# Act
Search[Seven Brief Lessons on Physics]

# Observation
Seven Brief Lessons on Physics (Italian: Sette brevi lezioni di fisica) is a short book by the Italian physicist Carlo Rovelli. Originally published in Italian in...

# Thought
The author of Seven Brief Lessons on Physics is Carlo Rovelli. I need to search Carlo Rovelli next and find when he has word in France since.

# Act
Search[Carlo Rovelli]

# Observation
Carlo Rovelli is an Italian theoretical physicist and writer who has worked in Italy, the United States and, since 2000, in France.[1] He is also currently a Distinguished Visiting Research Chair at the Perimeter Institute...

# Thought
Carlo Rovelli has worked in France since 2000. So the answer is 2000.

# Act
Finish[2000] user Great job! Now for the real question:
 How to calculate the limit of a fraction, when both the numerator and the denominator approach 0? assistant <span style='opacity: 1.0; display: inline; background-color: rgba(165, 165, 165, 0.1);' title='{{#geneach 'chain' stop=False~}}
# Thought
{{gen 'this.thought' max_tokens=512 stop="
# Act"}}
# Act
{{select 'this.act' options=['Search', 'Finish']}}[{{gen 'this.act_content' max_tokens=512 stop="]"}}]
{{#if this.act == 'Search'}}
# Observation
{{#block hidden=True~}}
{{set 'this.observation' (search query=this.act_content thought=this.thought)}}
{{/block~}}
{{this.observation}}
{{else}}
{{break}}
{{/if}}
{{/geneach}}'># Thought
 To calculate the limit of a fraction when both the numerator and the denominator approach 0, we need to use L'Hôpital's rule.
 
# Act
 Search [ L'Hôpital's rule ]
 
# Observation
 # L'Hôpital's rule
## Corollary
A simple but very useful consequence of L'Hopital's rule is a well-known criterion for differentiability. It states the following: suppose that f is continuous at a, and that f ′ ( x ) {\displaystyle f'(x)} exists for all x in some open interval containing a, except perhaps for x = a {\displaystyle x=a} . Suppose, moreover, that lim x → a f ′ ( x ) {\displaystyle \lim _{x\to a}f'(x)} exists. Then f ′ ( a ) {\displaystyle f'(a)} also exists and f ′ ( a ) = lim x → a f ′ ( x ) . {\displaystyle f'(a)=\lim _{x\to a}f'(x).} In particular, f' is also continuous at a. 
 
# Thought
 L'Hôpital's rule is used when we have a limit of a function that tends to 0, and we want to find the limit of the same function raised to a power or divided by another function that also tends to 0.
 
# Act
 Finish [ L'Hôpital's rule ]